In [ ]:
import csv
import cv2
import numpy as np

lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
k = 0

for line in lines:
    if (k > 0):
        for i in range(3):
            # Load images from center, left and right cameras
            source_path = line[i]
            tokens = source_path.split('/')
            filename = tokens[-1]
            local_path = "./data/IMG/" + filename
            image = cv2.imread(local_path)
            images.append(image)
        correction = 0.2
        measurement = float(line[3])

        # Steering adjustment for center images
        measurements.append(measurement)

        # Add correction for steering for left images
        measurements.append(measurement+correction)

        # Minus correction for steering for right images
        measurements.append(measurement-correction)
    k+=1
# Arrays to store additional images and measurements
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)

    # Flip images to reduce bias from anti-clockwise driving
    flipped_image = cv2.flip(image, 1)
    flipped_measurement = float(measurement) * -1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)

X_train = np.array(augmented_images[0:])
y_train = np.array(augmented_measurements[0:])   

In [2]:
import tensorflow as tf
# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D
from keras.models import Model
import matplotlib.pyplot as plt
import keras
print(tf.__version__)
model = Sequential()

# Normalizing lambda layer
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20),(0,0))))

# First convolutional layer
model.add(Convolution2D(6,5,5,activation='relu'))
model.add(MaxPooling2D())

# Second convolutional layer
model.add(Convolution2D(6,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())

# Fully-connected layers
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss = 'mse', optimizer ='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch = 5)
model.save('model.h5')

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

1.1.0


I:\Program Files\Anaconda3\envs\carnd-term1\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
I:\Program Files\Anaconda3\envs\carnd-term1\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


NameError: name 'X_train' is not defined